<a href="https://colab.research.google.com/github/civanescu/CSV-upload-parse/blob/main/My_SillyTavern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title <-- Tap this if you run on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the audio player to keep the tab alive. (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

Silly Tavern Interface

In [4]:
# @title 1. Setup Environment, Clone SillyTavern & Install Dependencies (Persistent Drive Install)
import os
import time
import re
from google.colab import drive
import shutil # Import shutil for directory operations

print("Mounting Google Drive for persistent storage...")
# Mount Google Drive. Use force_remount=True to ensure a clean mount each session.
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please check if you are logged in and authorized. Cannot proceed without Google Drive.")
    # It's crucial to stop if drive mounting fails
    exit(1) # Hard stop execution if drive mount fails

# Define the *SINGLE* Google Drive path for SillyTavern
# This is where the application code and all user data will live persistently
sillytavern_path = "/content/drive/MyDrive/SillyTavern" # *** This variable now points to Drive ***

# Ensure the SillyTavern directory exists in Google Drive
print(f"\nEnsuring SillyTavern directory exists in Google Drive: {sillytavern_path}")
# Use ! for shell command in Colab to create directory and parents (-p)
# This command is safe to run even if the directory already exists
!mkdir -p "$sillytavern_path"

# Change directory to the Google Drive location FIRST
# ALL subsequent operations in this cell will happen here
print(f"Changing current directory to {sillytavern_path}...")
# Use %cd magic command in Colab for changing directory
try:
    %cd "$sillytavern_path"
    print(f"Current directory is now: {os.getcwd()}")
except Exception as e:
     print(f"Error changing directory to {sillytavern_path}: {e}")
     print("Cannot proceed without being in the SillyTavern directory in Drive.")
     exit(1) # Hard stop if changing directory fails

# --- Cloning and Updating Logic ---
# Check for a key file (package.json) to determine if SillyTavern is cloned
if not os.path.exists('package.json'):
    print(f"\nSillyTavern files not found in {sillytavern_path}. Attempting to clone...")
    print("Note: If the directory is not empty, cloning might fail.")
    # Attempt to clone into the current directory (which is sillytavern_path)
    # Use the 'staging' branch
    # Capture output and check for fatal errors
    clone_result = !git clone -b staging https://github.com/SillyTavern/SillyTavern.git . 2>&1 # Capture stdout and stderr

    # Check if cloning was successful by looking for common failure messages or the presence of package.json
    if "fatal:" in "\n".join(clone_result):
        print("\nERROR: Git cloning failed!")
        print("".join(clone_result)) # Print the full error output
        print("\nReason: The directory might exist and is not empty, preventing cloning.")
        print(f"Please manually check and potentially clear the directory: {sillytavern_path}")
        print("Cannot proceed without successful cloning.")
        exit(1) # Hard stop if cloning fails

    # Verify package.json exists after cloning attempt
    if not os.path.exists('package.json'):
         print(f"\nERROR: Cloning completed, but package.json not found in {sillytavern_path}.")
         print("Cloning might have reported success but still failed to place files correctly.")
         print("Cannot proceed without SillyTavern files.")
         exit(1) # Hard stop if package.json is still missing

    print("Cloning complete and verified.")

else:
    print(f"\nSillyTavern files (package.json) found at {sillytavern_path}.")
    # Assu[...]
    print(f"Error writing to {config_path}: {e}")
    print("Config file modification failed.")
    exit(1) # Exit if config modification fails


print("\nEnsure Whitelist Mode is Disabled Cell Finished.")

# Change back to content directory (optional, not strictly necessary for this setup)
# os.chdir('/content')


Mounting Google Drive for persistent storage...
Mounted at /content/drive
Google Drive mounted successfully.

Ensuring SillyTavern directory exists in Google Drive: /content/drive/MyDrive/SillyTavern
Changing current directory to /content/drive/MyDrive/SillyTavern...
/content/drive/MyDrive/SillyTavern
Current directory is now: /content/drive/MyDrive/SillyTavern

SillyTavern files not found in /content/drive/MyDrive/SillyTavern. Attempting to clone...
Note: If the directory is not empty, cloning might fail.

ERROR: Git cloning failed!
fatal: destination path '.' already exists and is not an empty directory.

Reason: The directory might exist and is not empty, preventing cloning.
Please manually check and potentially clear the directory: /content/drive/MyDrive/SillyTavern
Cannot proceed without successful cloning.

ERROR: Cloning completed, but package.json not found in /content/drive/MyDrive/SillyTavern.
Cloning might have reported success but still failed to place files correctly.
Cann

Cell 2: Configure Google Drive Persistence
This cell sets up symbolic links so SillyTavern reads and writes key data folders to your Google Drive. It creates the necessary folders in Drive if they don't exist.

# **Whitelist** IP

In [5]:
# @title 2. Ensure Whitelist Mode and IP restriction are Disabled
# Note: Renumbered to Cell 2 as old Cell 2 is removed/merged into Cell 1.
import os

# Define the SillyTavern path (set in Cell 1 - using the Drive path)
# Ensure this matches the sillytavern_path variable set in Cell 1!
sillytavern_path = "/content/drive/MyDrive/SillyTavern" # *** Correctly use the Drive path ***
config_path = os.path.join(sillytavern_path, 'config.yaml')

print(f"Ensuring Whitelist Mode and IP restriction are disabled in {config_path} using Python...")

# Check if the config file exists
if not os.path.exists(config_path):
    print(f"\nWarning: config.yaml not found at {config_path}.")
    print("SillyTavern might create this file on first run. If the Forbidden error persists,")
    print("try running Cell 4 once (it might fail), then re-run THIS cell (Cell 2), and then run Cell 4 again.")
    # We cannot proceed with modification if the file doesn't exist
    # exit(0) # Don't exit, allow subsequent cells to potentially create it
else:
    try:
        # Read the existing content of the config file
        print(f"Reading existing configuration from {config_path}...")
        with open(config_path, 'r') as f:
            lines = f.readlines()

        modified_lines = []
        whitelist_modified = False
        restrict_modified = False

        # Iterate through the lines and modify the relevant ones
        print("Modifying 'whitelistMode' and 'restrict_ip' settings...")
        for line in lines:
            # Check for whitelistMode line (case-insensitive and flexible with spacing)
            if re.match(r'^\s*whitelistMode\s*:', line, re.IGNORECASE):
                # Replace the line with the desired setting
                modified_lines.append("whitelistMode: false\n")
                whitelist_modified = True
                print(f"  - Set 'whitelistMode' to false.")
            # Check for restrict_ip line (case-insensitive and flexible with spacing)
            elif re.match(r'^\s*restrict_ip\s*:', line, re.IGNORECASE):
                 # Replace the line with the desired setting
                 modified_lines.append("restrict_ip: false\n")
                 restrict_modified = True
                 print(f"  - Set 'restrict_ip' to false.")
            else:
                # Keep all other lines as they are
                modified_lines.append(line)

        # If the lines were not found, potentially add them (though they should exist in default config)
        if not whitelist_modified:
             print("Warning: 'whitelistMode' line not found in config. Adding it.")
             modified_lines.append("whitelistMode: false\n") # Add it at the end
        if not restrict_modified:
             print("Warning: 'restrict_ip' line not found in config. Adding it.")
             modified_lines.append("restrict_ip: false\n") # Add it at the end


        # Write the entire modified content back to the file
        print(f"Writing modified configuration back to {config_path}...")
        with open(config_path, 'w') as f:
            f.writelines(modified_lines)

        print("Configuration file updated successfully.")

        # --- Verification ---
        print("-" * 30)
        print(f"Verifying settings in {config_path}...")
        found_whitelist_setting = False
        found_restrict_setting = False
        try:
            with open(config_path, 'r') as f:
                for line in f:
                     if re.match(r'^\s*whitelistMode\s*:\s*false', line, re.IGNORECASE):
                         print("Verification: 'whitelistMode: false' found.")
                         found_whitelist_setting = True
                     if re.match(r'^\s*restrict_ip\s*:\s*false', line, re.IGNORECASE):
                          print("Verification: 'restrict_ip: false' found.")
                          found_restrict_setting = True
                     if found_whitelist_setting and found_restrict_setting:
                         break # Found both, can stop reading

            if not found_whitelist_setting:
                 print("Verification failed: 'whitelistMode: false' not found after modification.")
            if not found_restrict_setting:
                 print("Verification failed: 'restrict_ip: false' not found after modification.")

        except Exception as ve:
             print(f"Error during verification: {ve}")

        print("-" * 30)
        # ------------------------------------------------------------------

    except FileNotFoundError:
         # This case is already handled by the initial check, but good practice
         print(f"Error: config.yaml not found at {config_path} during read/write operation.")
         # exit(1) # Don't exit, file might be created later
    except Exception as e:
        print(f"Error processing {config_path}: {e}")
        print("Config file modification failed.")
        exit(1) # Exit if a general error occurs during processing


print("\nEnsure Whitelist Mode is Disabled Cell Finished.")

# The current directory should still be the SillyTavern directory in Drive
# from Cell 1's %cd command.


Ensuring Whitelist Mode and IP restriction are disabled in /content/drive/MyDrive/SillyTavern/config.yaml using Python...
Reading existing configuration from /content/drive/MyDrive/SillyTavern/config.yaml...
Modifying 'whitelistMode' and 'restrict_ip' settings...
  - Set 'whitelistMode' to false.
  - Set 'restrict_ip' to false.
Writing modified configuration back to /content/drive/MyDrive/SillyTavern/config.yaml...
Configuration file updated successfully.
------------------------------
Verifying settings in /content/drive/MyDrive/SillyTavern/config.yaml...
Verification: 'whitelistMode: false' found.
Verification: 'restrict_ip: false' found.
------------------------------

Ensure Whitelist Mode is Disabled Cell Finished.


In [6]:
# @title 3. Install Cloudflared
# Note: Renumbered to Cell 3. This cell installs the cloudflared executable.
import os
import subprocess

# Check if cloudflared is already installed
# Use 'which' command to find the path, check return code
install_path = "/usr/local/bin/cloudflared" # Standard installation path

print("Checking if cloudflared is already installed...")
if os.path.exists(install_path):
    print("cloudflared is already installed.")
else:
    print("cloudflared not found. Installing...")
    # Download and install cloudflared
    # Using -L follows redirects
    # Using -o specifies output file name
    # Using chmod +x makes the downloaded file executable
    try:
        !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
        !chmod +x /usr/local/bin/cloudflared
        print("cloudflared installed successfully.")
    except Exception as e:
        print(f"Error installing cloudflared: {e}")
        print("Cannot proceed without cloudflared.")
        exit(1) # Exit if cloudflared installation fails

print("\nCloudflared Installation Cell Finished.")


Checking if cloudflared is already installed...
cloudflared not found. Installing...
--2025-06-02 09:27:00--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.5.0/cloudflared-linux-amd64 [following]
--2025-06-02 09:27:00--  https://github.com/cloudflare/cloudflared/releases/download/2025.5.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/797840ed-70cb-47b8-a6fe-ecb4b3385c94?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250602%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250602T092558Z&X-Amz-Expires=300&X-A

Cell 5: Start SillyTavern Server and Cloudflare Tunnel
This cell starts the SillyTavern Node.js server and then starts the Cloudflare tunnel to expose it. It captures the tunnel output to find the public URL.

In [7]:
# @title 4. Start SillyTavern and Cloudflare Tunnel
# Note: Renumbered to Cell 4 as old Cell 2 and 3 are removed/merged.
import os
import time
import subprocess
import re
import sys # Import sys for detailed error info
import signal # Import signal to terminate process groups

# Define the SillyTavern path (set in Cell 1)
sillytavern_path = "/content/drive/MyDrive/SillyTavern" # *** Use the Drive path ***

# Change directory to the SillyTavern location in Drive
# This is crucial for start.sh or node server.js to run correctly
print(f"Changing current directory to {sillytavern_path} before starting...")
try:
    # Use %cd magic command in Colab for changing directory
    %cd "$sillytavern_path"
    print(f"Current directory is now: {os.getcwd()}")
except Exception as e:
    print(f"Error changing directory to {sillytavern_path}: {e}")
    print("Cannot start SillyTavern. Make sure Cell 1 completed successfully and Google Drive is mounted.")
    # It's critical to be in the correct directory, stop if it fails.
    exit(1) # Exit with a non-zero status code to indicate an error

print("\nStarting SillyTavern server...")
# Run the start.sh script in the background
try:
    # Command to execute start.sh
    # Using shell=True is necessary for Popen to execute a shell script correctly.
    # Pass the command as a single string when shell=True.
    start_command = "./start.sh"

    print(f"Executing start script: {start_command} in background...")
    # Start the process in the background.
    # preexec_fn=os.setsid is good for detaching the process.
    # stdout=subprocess.PIPE, stderr=subprocess.STDOUT capture output.
    # text=True decodes output.
    # Use shell=True to execute the script via the shell.
    server_process = subprocess.Popen(start_command, shell=True, preexec_fn=os.setsid,
                                      stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(f"SillyTavern server process started with PID: {server_process.pid}")

    # Give SillyTavern time to start up and open its port (default 3000)
    # The exact time needed can vary, 15 seconds is a reasonable starting point.
    print("Waiting 15 seconds for SillyTavern server to initialize...")
    time.sleep(15) # Increased sleep slightly just in case

    # Check if the server process is still running
    if server_process.poll() is not None:
        print(f"\nERROR: SillyTavern server process (PID: {server_process.pid}) exited prematurely with return code {server_process.poll()}")
        print("Check the output above for any errors from the start.sh script.")
        # Print captured output from the server process
        server_output, _ = server_process.communicate()
        if server_output:
            print("\nSillyTavern server output:")
            print(server_output.strip())
        exit(1) # Exit as server failed to start


    print("\nStarting Cloudflare tunnel...")
    # Run cloudflared tunnel --url http://localhost:3000
    # Capture output to find the public tunnel URL
    # Use the full path to cloudflared just in case /usr/local/bin isn't in PATH
    tunnel_command = ["/usr/local/bin/cloudflared", "tunnel", "--url", "http://localhost:3000"]

    print("Executing cloudflared tunnel command in background and capturing output...")
    # Start the tunnel process in the background.
    # Use Popen and read stdout to find the URL line.
    # The tunnel process needs to stay alive for access.
    cloudflared_process = subprocess.Popen(tunnel_command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(f"Cloudflared tunnel process started with PID: {cloudflared_process.pid}")

    tunnel_url = None
    print("Waiting for Cloudflare tunnel URL (timeout in 60 seconds)...")
    # Read output line by line to find the URL
    start_time = time.time()
    timeout = 60 # Wait up to 60 seconds for the URL
    # Use readline() in a loop with a timeout
    # Use iter with a sentinel for more robust reading
    for line in iter(cloudflared_process.stdout.readline, ''):
        # Check for timeout
        if time.time() - start_time > timeout:
            print("\nTimeout waiting for Cloudflare tunnel URL. Tunnel might have failed to start.")
            # Attempt to terminate processes cleanly if timed out
            # Check if server is still running and try to terminate
            if server_process.poll() is None:
                 print(f"Attempting to terminate server process group (PID: {server_process.pid})...")
                 try: os.killpg(os.getpgid(server_process.pid), signal.SIGTERM)
                 except OSError as e: print(f"Error terminating server process: {e}")
            # Check if tunnel is still running and try to terminate
            if cloudflared_process.poll() is None:
                 print(f"Attempting to terminate tunnel process group (PID: {cloudflared_process.pid})...")
                 try: os.killpg(os.getpgid(cloudflared_process.pid), signal.SIGTERM)
                 except OSError as e: print(f"Error terminating tunnel process: {e}")
            exit(1) # Exit due to timeout
            break # Exit the loop

        print(f"Cloudflared output: {line.strip()}") # Print output for debugging

        # Look for the URL pattern (e.g., https://something.cloudflarezero trust.com or trycloudflare.com)
        url_match = re.search(r'https?://[^.\s]+\.(?:cloudflarezero\.com|trycloudflare\.com)', line)
        if url_match:
            tunnel_url = url_match.group(0)
            print("\nCloudflare tunnel URL found!")
            break # Found the URL, exit the loop

    # Check if the loop finished because the process exited rather than timing out/finding URL
    if not tunnel_url and cloudflared_process.poll() is not None:
         print("\nCloudflare tunnel process exited unexpectedly before finding the URL.")
         print("Check the Colab runtime logs for cloudflared output above for errors.")
         # Attempt to print remaining output from the process if it exited
         remaining_output = cloudflared_process.stdout.read()
         if remaining_output:
             print("\nRemaining Cloudflared output:")
             print(remaining_output.strip())
         # Exit the script as the tunnel failed
         exit(1) # Exit with error code


    if tunnel_url:
        print("\n" + "="*60)
        print("                SILLYTAVERN INTERFACE IS READY!                 ")
        print("="*60)
        print(f" Access it at the following URL: \n\n {tunnel_url} \n")
        print("="*60)
        print("Important:")
        print(f"- Your SillyTavern installation and data (chats, characters, etc.) is saved to Google Drive at: {sillytavern_path}")
        print("- The Cloudflare tunnel and SillyTavern server will remain active as long as this cell is running.")
        print("- Closing the browser tab or stopping this cell will terminate them.")
        print("- You can check the cell output above for SillyTavern and Cloudflared logs.")
        print("="*60)
    else:
        print("\nFailed to get Cloudflare tunnel URL after startup and timeout.")
        print("Check the cell output above for Cloudflared error messages.")
        # Exit the script as the tunnel failed
        exit(1) # Exit with error code


except FileNotFoundError:
    print(f"Error: The start script ({start_command}) or cloudflared executable was not found.")
    print("Make sure Cell 1 and Cell 3 completed successfully.")
    exit(1) # Exit with error code
except Exception as e:
    print(f"An unexpected error occurred during startup: {e}")
    # Provide more detailed error info
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(f"Error Type: {exc_type}, File: {fname}, Line: {exc_tb.tb_lineno}")
    print("Startup process failed.")
    exit(1) # Exit with error code


print("\nSillyTavern and Tunnel Start Cell Finished (Note: Server and tunnel are running in background).")
# The cell output will stop updating here, but the processes continue.
# You can add print statements in the loop reading cloudflared output to see live logs.

# To keep the Colab cell 'active' and prevent the runtime from timing out
# while the background processes are running, you might need a long-running
# task at the end of the cell, like an infinite loop with sleep.
# However, Popen with running processes *usually* keeps the cell alive.
# If your session times out prematurely, consider adding this:
# try:
#     while True:
#         time.sleep(3600) # Sleep for an hour, keep session alive
# except KeyboardInterrupt:
#     print("Cell interrupted. Attempting to stop processes...")
#     # Add logic here to terminate server_process and cloudflared_process
#     pass # Simple placeholder


Changing current directory to /content/drive/MyDrive/SillyTavern before starting...
/content/drive/MyDrive/SillyTavern
Current directory is now: /content/drive/MyDrive/SillyTavern

Starting SillyTavern server...
Executing start script: ./start.sh in background...
SillyTavern server process started with PID: 627
Waiting 15 seconds for SillyTavern server to initialize...

ERROR: SillyTavern server process (PID: 627) exited prematurely with return code 127
Check the output above for any errors from the start.sh script.

SillyTavern server output:
/bin/sh: 1: ./start.sh: not found

Starting Cloudflare tunnel...
Executing cloudflared tunnel command in background and capturing output...
Cloudflared tunnel process started with PID: 694
Waiting for Cloudflare tunnel URL (timeout in 60 seconds)...
Cloudflared output: 2025-06-02T09:27:15Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these a